In [65]:
# Import libraries for data manipulation and ETL pipeline
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from pandarallel import pandarallel
pandarallel.initialize(verbose=0)
from tqdm import tqdm

In [66]:
# Establish API connection
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid = '3fda75b7146a4769b207ee44017b3abe'
secret = '2a755cb04a18406b9394dbef2f8069dd'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [67]:
# Connect to SQL Server
import sqlalchemy as sql
from sqlalchemy import text
import pyodbc
server = 'LAPTOP-V3754MEK' 
database = 'Spotify'

engine = sql.create_engine('mssql+pyodbc://' + server + '/' + database + '?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')
conn = engine.connect()
results = engine.execute("SELECT track_uri2 FROM dim_tracks ORDER BY track_uri2")
results = pd.Series(results)
results.head()

0    (spotify:track:0002yNGLtYSYtc0X6ZnFvp)
1    (spotify:track:00039MgrmLoIzSpuYKurn9)
2    (spotify:track:0003Z98F6hUq7XxqSRM87H)
3    (spotify:track:0004ExljAge0P5XWn1LXmW)
4    (spotify:track:0005rgjsSeVLp1cze57jIN)
dtype: object

In [68]:
# Extract tracks from API
output_df = pd.DataFrame()
for row in tqdm(results[425001:450001]):
      output = sp.audio_features(row)
      output_df = output_df.append(output)
output_df.head()

100%|██████████| 25000/25000 [36:15<00:00, 11.49it/s]  


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,0
0,0.564,0.486,1.0,-12.205,0.0,0.0387,0.00144,0.692000,0.0884,0.679,149.920,audio_features,1JaocWtKVCDiRERQpDh8s3,spotify:track:1JaocWtKVCDiRERQpDh8s3,https://api.spotify.com/v1/tracks/1JaocWtKVCDi...,https://api.spotify.com/v1/audio-analysis/1Jao...,197333.0,4.0,NaN
0,0.742,0.818,5.0,-3.954,0.0,0.2660,0.00253,0.000000,0.2210,0.519,78.969,audio_features,1jAOdMcFKekgKXB21XsnJr,spotify:track:1jAOdMcFKekgKXB21XsnJr,https://api.spotify.com/v1/tracks/1jAOdMcFKekg...,https://api.spotify.com/v1/audio-analysis/1jAO...,172932.0,4.0,NaN
0,0.600,0.596,4.0,-6.706,0.0,0.0274,0.04220,0.024600,0.1410,0.257,126.045,audio_features,1JaOepr3SJa7zhQgl4CfLz,spotify:track:1JaOepr3SJa7zhQgl4CfLz,https://api.spotify.com/v1/tracks/1JaOepr3SJa7...,https://api.spotify.com/v1/audio-analysis/1JaO...,216507.0,4.0,NaN
0,0.343,0.636,0.0,-10.703,1.0,0.0674,0.42700,0.009130,0.1430,0.554,155.842,audio_features,1JaOhdWitLJB5bcXR3T9tl,spotify:track:1JaOhdWitLJB5bcXR3T9tl,https://api.spotify.com/v1/tracks/1JaOhdWitLJB...,https://api.spotify.com/v1/audio-analysis/1JaO...,820733.0,4.0,NaN
0,0.692,0.147,2.0,-16.150,1.0,0.0431,0.74200,0.000001,0.0938,0.763,96.083,audio_features,1jAoJzGETQaulxFXcaN2RG,spotify:track:1jAoJzGETQaulxFXcaN2RG,https://api.spotify.com/v1/tracks/1jAoJzGETQau...,https://api.spotify.com/v1/audio-analysis/1jAo...,150013.0,4.0,NaN


In [69]:
# Load data into SQL Server
output_df = output_df[['uri', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']]
output_df = output_df.rename(columns={'key':'music_key', 'mode':'modal'})
output_df.to_sql("audio_features2", engine, if_exists='append', index=False)
print(f'{len(output_df)} rows imported to database.')
new_query = engine.execute('SELECT COUNT(*) FROM audio_features2')
new_query = pd.Series(new_query)
print(f'App contains {new_query[0]} songs.')

25000 rows imported to database.
App contains (450001,) songs.
